In [90]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
from torch.nn import functional as F
from sklearn.metrics import roc_auc_score

from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import sys
DIR='/content/drive/My\ Drive/pan/'
sys.path.append(DIR)

device = 'cuda' if torch.cuda.is_available() else 'cpu' 
print('Using {} device'.format(device))     

Mounted at /content/drive
Using cuda device


In [91]:
data = pd.read_csv('/content/drive/My Drive/pan/USAir.csv')


In [ ]:
num_epochs = 100
batch_size = 10
learning_rate = 0.0001

In [ ]:
data = pd.read_csv('/content/drive/My Drive/pan/USAir.csv')


train_data = np.array(data.iloc[:,0:8])
input_size = np.size(train_data,1)


train_labels = np.array(data.iloc[:,-2])
test_labels = np.array(data.iloc[:,-1])
pos0 = np.where((train_labels==0))
pos1 = np.where((train_labels==1))
len0=len(pos0[0])
len1=len(pos1[0])

train_labels0=train_labels[pos0]
train_labels1=train_labels[pos1]
train_data0=train_data[pos0,:]
train_data0 = train_data0.reshape(np.size(train_data0,1),np.size(train_data0,2))
train_data1=train_data[pos1,:]
train_data1 = train_data1.reshape(np.size(train_data1,1),np.size(train_data1,2))

pos = np.where((train_labels==0))
test_data = train_data[pos,:]
test_labels = test_labels[pos]
test_data = test_data.reshape(np.size(test_data,1),np.size(test_data,2))


x=train_data1
y=train_labels1

print(len0,len1)
for i in range(int(1.0*len0/len1)):
    train_data1=np.concatenate([train_data1,x])
    train_labels1=np.concatenate([train_labels1,y])

train_data0 = torch.from_numpy(train_data0.astype(np.float32)).to(device)
train_labels0 = torch.from_numpy(train_labels0.astype(np.float32)).to(device)
train_data1 = torch.from_numpy(train_data1.astype(np.float32)).to(device)
train_labels1 = torch.from_numpy(train_labels1.astype(np.float32)).to(device)
test_data = torch.from_numpy(test_data.astype(np.float32)).to(device)
test_labels = torch.from_numpy(test_labels.astype(np.float32)).to(device)

train_dataset0 = Data.TensorDataset(train_data0, train_labels0)
train_dataset1 = Data.TensorDataset(train_data1, train_labels1)
test_dataset = Data.TensorDataset(test_data, test_labels)

train_loader0 = Data.DataLoader(
    dataset = train_dataset0,
    batch_size = batch_size,
    shuffle = False,
)

train_loader1 = Data.DataLoader(
    dataset = train_dataset1,
    batch_size = batch_size,
    shuffle = False,
)


test_loader = Data.DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = False,
)




class LogisticRegression(torch.nn.Module):
    def __init__(self,input_size):
        super(LogisticRegression, self).__init__()
        self.linear1 = torch.nn.Linear(input_size,300)
        self.linear2 = torch.nn.Linear(300,100)
        self.linear3 = torch.nn.Linear(100,1)
        self.act1= nn.ReLU()
        self.act2= nn.ReLU()
        

    def forward(self, x):
        out= self.linear1(x)
        out = self.act1(out)
        out= self.linear2(out)
        out = self.act2(out)
        out = self.linear3(out)
        out = torch.sigmoid(out)
        return out




model = LogisticRegression(input_size).to(device)
criterion = nn.BCELoss(size_average = True)
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print (name,param)

#criterion=nn.L1Loss()
#criterion=nn.MSELoss()

# for i, (stats0, labels0) in  enumerate(train_loader0):
#     print(i)

# for i, (stats0, labels0) in  enumerate(train_loader1):
#     print(i)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.99)

test_labels = test_labels.cpu().detach().numpy()
total_step = len(train_loader0)
for epoch in range(num_epochs):
    for (i, (stats0, labels0)),(ii, (stats1, labels1)) in zip (enumerate(train_loader0),enumerate(train_loader1)):
        stats=torch.cat([stats0,stats1])
        labels=torch.cat([labels0,labels1])
        
        stats = stats.reshape(-1, input_size)
        
        outputs = model(stats)
        
        outputs = outputs.reshape(-1)
        print(outputs.shape)
        print(labels.shape)
        loss = criterion(outputs,labels)

        with torch.no_grad():
            if (i+1)%10== 0 or (i<10 and epoch==0) :
                #print('weight', model.linear.weight)
                #print('bias', model.linear.bias)
                test_scores = model(test_data).cpu().detach().numpy()
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, auc: {:.4f}'\
                    .format(epoch+1, num_epochs,i+1, total_step, loss.item(), roc_auc_score(test_labels, test_scores)))


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()



#test_labels = test_labels.cpu().detach().numpy()
#test_scores = model(test_data).cpu().detach().numpy()

auc = roc_auc_score(test_labels, test_scores)
print('Accuracy of the model on the test set: {} %'.format(auc))

53033 1912
torch.Size([20])
torch.Size([20])
Epoch [1/100], Step [1/5304], Loss: 0.6938, auc: 0.1285
torch.Size([20])
torch.Size([20])
Epoch [1/100], Step [2/5304], Loss: 0.6938, auc: 0.1286
torch.Size([20])
torch.Size([20])
Epoch [1/100], Step [3/5304], Loss: 0.6939, auc: 0.1284
torch.Size([20])
torch.Size([20])
Epoch [1/100], Step [4/5304], Loss: 0.6938, auc: 0.1283
torch.Size([20])
torch.Size([20])
Epoch [1/100], Step [5/5304], Loss: 0.6938, auc: 0.1285
torch.Size([20])
torch.Size([20])
Epoch [1/100], Step [6/5304], Loss: 0.6938, auc: 0.1288
torch.Size([20])
torch.Size([20])
Epoch [1/100], Step [7/5304], Loss: 0.6938, auc: 0.1286
torch.Size([20])
torch.Size([20])
Epoch [1/100], Step [8/5304], Loss: 0.6938, auc: 0.1287
torch.Size([20])
torch.Size([20])
Epoch [1/100], Step [9/5304], Loss: 0.6938, auc: 0.1288
torch.Size([20])
torch.Size([20])
Epoch [1/100], Step [10/5304], Loss: 0.6939, auc: 0.1284
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
tor

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
Epoch [1/100], Step [20/5304], Loss: 0.6938, auc: 0.1286
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
Epoch [1/100], Step [30/5304], Loss: 0.6938, auc: 0.1285
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
to

KeyboardInterrupt: ignored

In [ ]:
outputs-labels

tensor([ 0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  0.5000,
         0.5000,  0.5000, -0.5001, -0.5001, -0.5000, -0.5001, -0.5000, -0.5001,
        -0.5001, -0.5001, -0.5001, -0.5001], device='cuda:0',
       grad_fn=<SubBackward0>)

In [ ]:
labels[1]

tensor(0., device='cuda:0')

In [ ]:
for i in range(len(test_labels)):
    print(test_labels[i])


Streaming output truncated to the last 5000 lines.
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0

KeyboardInterrupt: ignored

In [ ]:
test_labels.sum()

tensor(213., device='cuda:0')

In [ ]:
test_scores

array([[0.04473177],
       [0.04473177],
       [0.04475073],
       ...,
       [0.04438842],
       [0.04468346],
       [0.04437087]], dtype=float32)

In [ ]:

auc = roc_auc_score(test_labels, test_labels)
auc

1.0

In [ ]:
train_loader = Data.DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True,
)

test_loader = Data.DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = False,
)

In [ ]:
test_data.shape.

torch.Size([1, 53033, 8])

In [ ]:
train_data = torch.from_numpy(train_data.astype(np.float32))
train_labels = torch.from_numpy(train_labels.astype(np.float32))
test_data = torch.from_numpy(test_data.astype(np.float32))
test_labels = torch.from_numpy(test_labels.astype(np.float32))

In [ ]:
train_data

tensor([[ 1.0991e-04,  2.1608e-05,  5.3762e-06,  ...,  3.1228e-07,
          3.1005e-07,  3.2626e-07],
        [ 2.1985e-04,  2.1608e-05,  5.7795e-06,  ...,  3.4586e-07,
          3.4408e-07,  3.6233e-07],
        [ 1.0991e-04,  2.1608e-05,  5.3762e-06,  ...,  3.1228e-07,
          3.1005e-07,  3.2626e-07],
        ...,
        [ 1.7764e-15,  1.2213e-05,  5.3762e-07,  ...,  2.6962e-07,
          2.8670e-07,  3.1342e-07],
        [ 1.0991e-04,  1.0334e-05,  1.0752e-06,  ...,  9.5517e-10,
          1.1501e-10,  4.9937e-11],
        [-1.7764e-15,  6.5763e-06,  2.6881e-07,  ...,  1.7347e-07,
          1.8631e-07,  2.0415e-07]])

In [ ]:
roc_auc_score([1,2,3],[1,2,3])

ValueError: ignored

In [ ]:
train_data

array([[ 1.09908227e-04,  2.16075957e-05,  5.37620391e-06, ...,
         3.12280331e-07,  3.10051142e-07,  3.26257961e-07],
       [ 2.19852700e-04,  2.16080626e-05,  5.77945143e-06, ...,
         3.45864247e-07,  3.44081954e-07,  3.62333438e-07],
       [ 1.09908227e-04,  2.16075957e-05,  5.37620390e-06, ...,
         3.12280321e-07,  3.10051128e-07,  3.26257947e-07],
       ...,
       [ 1.77635684e-15,  1.22130463e-05,  5.37621691e-07, ...,
         2.69620507e-07,  2.86703767e-07,  3.13424188e-07],
       [ 1.09908227e-04,  1.03341258e-05,  1.07524309e-06, ...,
         9.55171942e-10,  1.15008447e-10,  4.99369435e-11],
       [-1.77635684e-15,  6.57627421e-06,  2.68810879e-07, ...,
         1.73472756e-07,  1.86310672e-07,  2.04148826e-07]])

In [ ]:

train_data = np.array(data.iloc[:,0:8])


train_data[:1000,:].shape

(1000, 8)

In [ ]:
train_data = np.array(data.iloc[:,0:8])
train_data.shape

(54945, 8)

In [ ]:
train_data=[[1,1],[2,2]]

In [ ]:
train_data=(train_data-np.mean(train_data,axis=1).reshape(-1,1))

In [ ]:
train_data

array([[0., 0.],
       [0., 0.]])

In [ ]:
train_data=(train_data-np.mean(train_data,axis=1).reshape(-1,1))
train_data=train_data/(np.amax(np.abs(train_data),axis=1).reshape(-1,1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [103]:
num_epochs = 100
batch_size = 1000
learning_rate = 0.001


train_data = np.array(data.iloc[:,0:8])
train_data=np.log(np.abs(train_data)*10000+0.000000000001)
input_size = np.size(train_data,1)
train_labels = np.array(data.iloc[:,-2])
test_labels = np.array(data.iloc[:,-1])
pos = np.where((train_labels==0))

class_counts = [np.size(pos,1),np.size(train_data,0)-np.size(pos,1)]
num_samples = sum(class_counts)
class_weights = [num_samples/class_counts[i] for i in range(len(class_counts))]
weights = [class_weights[train_labels[i]] for i in range(int(num_samples))]
sampler = torch.utils.data.sampler.WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))


test_data = train_data[pos,:]
test_labels = test_labels[pos]
test_data = test_data.reshape(np.size(test_data,1),np.size(test_data,2))

train_data = torch.from_numpy(train_data.astype(np.float32)).to(device)
train_labels = torch.from_numpy(train_labels.astype(np.float32)).to(device)
test_data = torch.from_numpy(test_data.astype(np.float32)).to(device)
test_labels = torch.from_numpy(test_labels.astype(np.float32)).to(device)

print(train_labels.size())

train_dataset = Data.TensorDataset(train_data, train_labels)
test_dataset = Data.TensorDataset(test_data, test_labels)




train_loader = Data.DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    #shuffle = True,
    sampler = sampler,
)

test_loader = Data.DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = False,
)

class LogisticRegression(torch.nn.Module):
    def __init__(self,input_size):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_size, 1,bias=True)     

    def forward(self, x):
        out = torch.sigmoid(self.linear(x))
        #out = self.linear(x)
        return out


# class LogisticRegression(torch.nn.Module):
#     def __init__(self,input_size):
#         super(LogisticRegression, self).__init__()
#         self.linear1 = torch.nn.Linear(input_size,10000)
#         self.linear2 = torch.nn.Linear(10000,1000)
#         self.linear3 = torch.nn.Linear(1000,800)
#         self.linear4 = torch.nn.Linear(800,200)
#         self.linear5 = torch.nn.Linear(200,1)
#         self.act1= nn.ReLU()
#         self.act2= torch.sin
#         self.act3= torch.sin
#         self.act4= nn.ReLU()

#     def forward(self, x):
#         out= self.linear1(x)
#         out = self.act1(out)
#         out= self.linear2(out)
#         out = self.act2(out)
#         out = self.linear3(out)
#         out = self.act3(out)
#         out = self.linear4(out)
#         out = self.act4(out)
#         out = self.linear5(out)
#         out = torch.sigmoid(out)
#         return out


model = LogisticRegression(input_size).to(device)
criterion = nn.BCEWithLogitsLoss()
#criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.99)
total_step = len(train_loader)
test_labels = test_labels.cpu().detach().numpy()
for epoch in range(num_epochs):
    # if epoch%100==0:
    #     learning_rate=learning_rate/2
    #     optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for i, (stats, labels) in enumerate(train_loader):
        stats = stats.reshape(-1, input_size).to(device)
        outputs = model(stats)
        outputs = outputs.reshape(-1).to(device)
        loss = criterion(outputs,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i)%1 == 0:
            test_scores = model(test_data).cpu().detach().numpy()
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, auc: {:.4f}'\
                .format(epoch+1, num_epochs,i+1, total_step, loss.item(), roc_auc_score(test_labels, test_scores)))
        if i==30:
            break


#test_labels = test_labels.cpu().detach().numpy()
#test_scores = model(test_data).cpu().detach().numpy()
#auc = roc_auc_score(test_labels, test_scores)
#print('Accuracy of the model on the test set: {} '.format(auc))

torch.Size([54945])
Epoch [1/100], Step [1/55], Loss: 0.6293, auc: 0.8860
Epoch [1/100], Step [2/55], Loss: 0.6421, auc: 0.8874
Epoch [1/100], Step [3/55], Loss: 0.6327, auc: 0.8894
Epoch [1/100], Step [4/55], Loss: 0.6421, auc: 0.8920
Epoch [1/100], Step [5/55], Loss: 0.6333, auc: 0.8950
Epoch [1/100], Step [6/55], Loss: 0.6389, auc: 0.8985
Epoch [1/100], Step [7/55], Loss: 0.6406, auc: 0.9024
Epoch [1/100], Step [8/55], Loss: 0.6338, auc: 0.9066
Epoch [1/100], Step [9/55], Loss: 0.6346, auc: 0.9110
Epoch [1/100], Step [10/55], Loss: 0.6361, auc: 0.9152
Epoch [1/100], Step [11/55], Loss: 0.6302, auc: 0.9192
Epoch [1/100], Step [12/55], Loss: 0.6369, auc: 0.9226
Epoch [1/100], Step [13/55], Loss: 0.6280, auc: 0.9259
Epoch [1/100], Step [14/55], Loss: 0.6248, auc: 0.9286
Epoch [1/100], Step [15/55], Loss: 0.6277, auc: 0.9308
Epoch [1/100], Step [16/55], Loss: 0.6358, auc: 0.9327
Epoch [1/100], Step [17/55], Loss: 0.6254, auc: 0.9341
Epoch [1/100], Step [18/55], Loss: 0.6201, auc: 0.9352

In [ ]:
test_labels

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
labels

tensor([0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
        0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
        1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
        1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0.,
        1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0.,
        0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0.,
        0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1.,
        1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0.,
        1., 1., 0., 1., 0., 0., 0., 0., 

In [ ]:
stats

tensor([[-21.7553,   2.2300,   2.1056,  ...,   2.6174,   2.7364,   2.8426],
        [-21.7553,   1.4416,   0.6809,  ...,   1.1771,   1.2808,   1.3844],
        [-27.6310,  -0.1678,  -2.5177,  ...,  -3.4393,  -3.4003,  -3.3274],
        ...,
        [  3.0904,   0.7705,  -0.5483,  ...,  -3.3643,  -3.3695,  -3.3178],
        [-21.7553,   2.9079,   2.6908,  ...,   3.0970,   3.2070,   3.3096],
        [  2.3972,   1.6423,  -0.1351,  ...,  -0.7345,  -0.6635,  -0.5722]],
       device='cuda:0')